In [18]:
import os
from pathlib import Path
import re
import json
from collections import defaultdict, OrderedDict
import pprint
import docx
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup
import markdown
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [19]:
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)

True

## Preprocesses a collection of at least 10 legal documents . Apply tokenization, stop word removal, and stemming or lemmatization

In [20]:
lemmatizer = WordNetLemmatizer() 
pp = pprint.PrettyPrinter(width=140)

def print_doc_summary(doc, tokens_preview=30):
    """Print a compact, readable summary of a preprocessed document dict."""
    print('\n' + '='*100)
    print(f"Document: {doc.get('doc_id')}")
    print('-'*100)
    print(f"Path: {doc.get('path')}")
    print(f"Tokens: {len(doc.get('tokens', []))}, Sentences: {len(doc.get('sentences', []))}, Paragraphs: {len(doc.get('paragraphs', []))}")
    print()
    tokens = doc.get('tokens', [])
    lemmas = doc.get('lemmas', [])
    preview = ' '.join(tokens[:tokens_preview])
    preview_lemmas = ' '.join(lemmas[:tokens_preview])
    print('Tokens preview:')
    print(preview)
    print()
    print('Lemmas preview:')
    print(preview_lemmas)
    print('='*100 + '\n')

In [21]:
# Part 2: File reading utilities
def read_txt(path):
    return Path(path).read_text(encoding='utf8', errors='ignore')

def read_docx(path):
    doc = docx.Document(path)
    return "\n".join(p.text for p in doc.paragraphs)

def read_pdf(path):
    text_parts = []
    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            txt = page.extract_text()
            if txt:
                text_parts.append(txt)
    return "\n".join(text_parts)

def read_csv(path):
    try:
        df = pd.read_csv(path, encoding='utf8', dtype=str, keep_default_na=False)
        rows = []
        for _, row in df.iterrows():
            rows.append(" ".join([str(x) for x in row.values if str(x).strip()]))
        return "\n".join(rows)
    except Exception:
        return read_txt(path)

READERS = {
    ".txt": read_txt,
    ".md": read_txt,
    ".docx": read_docx,
    ".pdf": read_pdf,
    ".csv": read_csv
}

def read_file_auto(path):
    p = Path(path)
    reader = READERS.get(p.suffix.lower(), read_txt)
    return reader(path)

In [22]:
# Part 3: Preprocessing helpers and pipeline
STOPWORDS = set(stopwords.words('english'))
TOKEN_RE = re.compile(r"\w+")  # word tokens (alphanumeric)

In [23]:
def tokenize_text(text):
    return TOKEN_RE.findall(text.lower())

def lemmatize_list(tokens):
    return [lemmatizer.lemmatize(t) for t in tokens]

def preprocess_document(path, doc_id=None):
    """
    Reads file, splits into paragraphs, sentences, full tokens, and builds position metadata:
    returns dict with keys: doc_id, text, paragraphs, sentences, tokens, lemmas, pos_to_sent, pos_to_para
    """
    text = read_file_auto(path)
    paragraphs = [p.strip() for p in re.split(r'\n\s*\n', text) if p.strip()]
    if not paragraphs:
        paragraphs = [text.strip()]

    # Build sentences list while maintaining paragraph boundaries:
    sentences = []
    sent_to_para = []  # map sentence index -> paragraph index
    for p_idx, para in enumerate(paragraphs):
        sents = sent_tokenize(para)
        for s in sents:
            sentences.append(s)
            sent_to_para.append(p_idx)

    tokens = tokenize_text(text)            # original normalized tokens (lowercased)
    lemmas = lemmatize_list(tokens)        # lemmatized tokens aligned with tokens

    # Map token positions to sentence index and paragraph index.
    pos_to_sent = {}   # token_position -> sentence_index
    pos_to_para = {}   # token_position -> paragraph_index
    token_index = 0
    for s_idx, s in enumerate(sentences):
        s_tokens = tokenize_text(s)
        for _ in s_tokens:
            # assign mapping for this token position
            pos_to_sent[token_index] = s_idx
            pos_to_para[token_index] = sent_to_para[s_idx]
            token_index += 1
    # If mismatch, we still keep mappings for positions we could assign.

    if doc_id is None:
        doc_id = Path(path).name

    return {
        "doc_id": doc_id,
        "path": str(path),
        "text": text,
        "paragraphs": paragraphs,
        "sentences": sentences,
        "tokens": tokens,
        "lemmas": lemmas,
        "pos_to_sent": pos_to_sent,
        "pos_to_para": pos_to_para
    }


## Construct a Positional inverted index that maps each term to document IDs and positional occurrences.(2)

In [24]:
# Part 4: Build positional inverted index

def build_positional_inverted_index(docs_preprocessed):
    """
    docs_preprocessed: list of dicts returned by preprocess_document
    Returns: inverted_index: dict { term: { doc_id: [positions] } }
    """
    index = defaultdict(lambda: defaultdict(list))
    for doc in docs_preprocessed:
        doc_id = doc['doc_id']
        lemmas = doc['lemmas']
        for pos, lemma in enumerate(lemmas):
            if lemma in STOPWORDS:
                continue
            index[lemma][doc_id].append(pos)
    # Convert postings lists to sorted lists
    for term in list(index.keys()):
        for doc_id in index[term]:
            index[term][doc_id] = sorted(index[term][doc_id])
    return index

def sorted_inverted_index_repr(index):
    return OrderedDict(sorted(((term, dict(sorted(postings.items()))) for term, postings in index.items()), key=lambda x: x[0]))

## Support Proximity-Based Search Operators - Implement query functionality that supports the following proximity search operators:
#### term1 /n term2: term1 and term2 must appear within n words of each other.
#### term1 /s term2: term1 and term2 must appear in the same sentence.
#### term1 /p term2: term1 and term2 must appear in the same paragraph.
##### Display Matching document IDs and Matching snippet text where the terms occur together.

In [25]:
# Part 5: Proximity and snippet utilities

def snippet_from_sent_idx(doc, sent_idx):
    if 0 <= sent_idx < len(doc['sentences']):
        return doc['sentences'][sent_idx].strip()
    return ""

def snippet_from_para_idx(doc, para_idx):
    if 0 <= para_idx < len(doc['paragraphs']):
        return doc['paragraphs'][para_idx].strip()
    return ""

def build_highlighted_snippet(doc, pos_pair, window_tokens=8, bg_color='#fff59d'):
    """
    Build an HTML snippet around pos_pair (a,b). Returns (highlighted_html, sentence_index, paragraph_index, start_token_pos)
    """
    tokens = doc['tokens']
    a, b = pos_pair
    a0, b0 = min(a,b), max(a,b)
    start = max(0, a0 - window_tokens)
    end = min(len(tokens), b0 + window_tokens + 1)
    snippet_tokens = tokens[start:end]
    local_a = a0 - start
    local_b = local_a + (b0 - a0) + 1
    def esc(s):
        return s.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
    pre = ' '.join(esc(t) for t in snippet_tokens[:local_a])
    phrase_text = ' '.join(esc(t) for t in snippet_tokens[local_a:local_b])
    post = ' '.join(esc(t) for t in snippet_tokens[local_b:])
    highlighted = f"{pre} <span style=\"background-color:{bg_color};color:#000;padding:2px 4px;border-radius:3px\">{phrase_text}</span> {post}".strip()
    s_idx = doc.get('pos_to_sent', {}).get(a0)
    p_idx = doc.get('pos_to_para', {}).get(a0)
    return highlighted, s_idx, p_idx, a0

def proximity_within_n(index, term1, term2, n):
    """
    term1 /n term2: return list of (doc_id, pairs_of_positions, snippet)
    Uses two-pointer algorithm for positions lists.
    """
    t1 = term1.lower()
    t2 = term2.lower()
    results = []
    postings1 = index.get(t1, {})
    postings2 = index.get(t2, {})
    common_docs = set(postings1.keys()) & set(postings2.keys())
    for doc_id in common_docs:
        p1 = postings1[doc_id]
        p2 = postings2[doc_id]
        i, j = 0, 0
        pairs = []
        # two-pointer to find all position pairs whose distance <= n
        while i < len(p1) and j < len(p2):
            a, b = p1[i], p2[j]
            if a == b:
                # same position (rare), skip
                if a < b:
                    i += 1
                else:
                    j += 1
                continue
            if abs(a - b) <= n:
                pairs.append((a, b))
                # advance the smaller pointer to seek other combos
                if a < b:
                    i += 1
                else:
                    j += 1
            else:
                if a < b:
                    i += 1
                else:
                    j += 1
        if pairs:
            results.append((doc_id, pairs))
    return results

def proximity_same_sentence(index, docs_by_id, term1, term2):
    """
    term1 /s term2: check if both lemmas exist in the same sentence.
    We'll use pos_to_sent mapping to determine sentence membership.
    Returns list of (doc_id, sent_index, snippet)
    """
    t1 = term1.lower()
    t2 = term2.lower()
    results = []
    postings1 = index.get(t1, {})
    postings2 = index.get(t2, {})
    common_docs = set(postings1.keys()) & set(postings2.keys())
    for doc_id in common_docs:
        doc = docs_by_id[doc_id]
        # build set of sentence indices containing term1 and term2
        sent_idxs_t1 = { doc['pos_to_sent'].get(pos) for pos in postings1[doc_id] }
        sent_idxs_t2 = { doc['pos_to_sent'].get(pos) for pos in postings2[doc_id] }
        # intersection ignoring None
        intersect = {s for s in sent_idxs_t1 if s is not None} & {s for s in sent_idxs_t2 if s is not None}
        for sidx in sorted(intersect):
            # find a pos for term1 and term2 within sentence sidx
            pos1 = next((pos for pos in postings1[doc_id] if doc['pos_to_sent'].get(pos)==sidx), None)
            pos2 = next((pos for pos in postings2[doc_id] if doc['pos_to_sent'].get(pos)==sidx), None)
            # choose a representative pair if both found
            if pos1 is not None and pos2 is not None:
                # pick smaller-first ordering
                pair = (pos1, pos2)
                results.append((doc_id, sidx, pair))
            else:
                # fallback: return sidx without positions
                results.append((doc_id, sidx, None))
    return results

def proximity_same_paragraph(index, docs_by_id, term1, term2):
    """
    term1 /p term2: check if both lemmas exist in the same paragraph.
    """
    t1 = term1.lower()
    t2 = term2.lower()
    results = []
    postings1 = index.get(t1, {})
    postings2 = index.get(t2, {})
    common_docs = set(postings1.keys()) & set(postings2.keys())
    for doc_id in common_docs:
        doc = docs_by_id[doc_id]
        para_idxs_t1 = { doc['pos_to_para'].get(pos) for pos in postings1[doc_id] }
        para_idxs_t2 = { doc['pos_to_para'].get(pos) for pos in postings2[doc_id] }
        intersect = {p for p in para_idxs_t1 if p is not None} & {p for p in para_idxs_t2 if p is not None}
        for pidx in sorted(intersect):
            pos1 = next((pos for pos in postings1[doc_id] if doc['pos_to_para'].get(pos)==pidx), None)
            pos2 = next((pos for pos in postings2[doc_id] if doc['pos_to_para'].get(pos)==pidx), None)
            if pos1 is not None and pos2 is not None:
                results.append((doc_id, pidx, (pos1, pos2)))
            else:
                results.append((doc_id, pidx, None))
    return results

def extract_snippet_from_positions(doc, pos_pair, window_tokens=8):
    tokens = doc['tokens']
    a, b = pos_pair
    start = max(0, min(a, b) - window_tokens)
    end = min(len(tokens), max(a, b) + window_tokens + 1)
    return " ".join(tokens[start:end])

## Implement Phrase Query Support - that:
##### ·        Accepts a query like "right to counsel" or "due process"

##### ·        Returns documents where all terms in the phrase appear in the same order and consecutively

In [26]:
# Part 6: Phrase query using positional index

def phrase_query(index, docs_by_id, phrase, window_tokens=8):
    """
    phrase: string like "right to counsel"
    Steps:
     - tokenize+lemmatize phrase terms (using same pipeline)
     - for candidate docs that contain first term, scan positions to see if consecutive positions match phrase lemmas
    Returns: list of (doc_id, start_pos, sentence_index, paragraph_index, highlighted_snippet_html)
    """
    phrase_tokens = [t for t in TOKEN_RE.findall(phrase.lower()) if t not in STOPWORDS]
    if not phrase_tokens:
        return []
    phrase_lemmas = [lemmatizer.lemmatize(t) for t in phrase_tokens]
    first = phrase_lemmas[0]
    results = []
    postings_first = index.get(first, {})
    L = len(phrase_lemmas)
    for doc_id, positions in postings_first.items():
        doc = docs_by_id[doc_id]
        lemmas = doc['lemmas']
        tokens = doc['tokens']
        for p in positions:
            # check bounds
            if p + L > len(lemmas):
                continue
            ok = True
            for offset in range(L):
                if lemmas[p + offset] != phrase_lemmas[offset]:
                    ok = False
                    break
            if ok:
                # found exact consecutive phrase starting at p
                s_idx = doc['pos_to_sent'].get(p)
                para_idx = doc['pos_to_para'].get(p)
                # build a token-window snippet and highlight the phrase tokens
                start = max(0, p - window_tokens)
                end = min(len(tokens), p + L + window_tokens)
                snippet_tokens = tokens[start:end]
                a = p - start
                b = a + L
                def esc(s):
                    return s.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                pre = ' '.join(esc(t) for t in snippet_tokens[:a])
                phrase_text = ' '.join(esc(t) for t in snippet_tokens[a:b])
                post = ' '.join(esc(t) for t in snippet_tokens[b:])
                highlighted = f"{pre} <span style=\"background-color:#fff59d;color:#000; padding:2px 4px; border-radius:3px\">{phrase_text}</span> {post}".strip()
                results.append((doc_id, p, s_idx, para_idx, highlighted))
                break
    return results

In [27]:
DATA_DIR = Path("./data")
files = sorted([p for p in DATA_DIR.iterdir() if p.is_file()])
print('\nFiles found: {}\n'.format(len(files)))
for i, f in enumerate(files, 1):
    try:
        size = f.stat().st_size
    except Exception:
        size = 0
    print(f"{i:2d}. {f.name}  ({f.suffix})  size={size} bytes")

files_list = files


Files found: 9

 1. BenchOpinion.md  (.md)  size=5149 bytes
 2. indian_supreme_court_judgments.csv  (.csv)  size=32277049 bytes
 3. Martin_v_Franklin_Capital_Corp.md  (.md)  size=5224 bytes
 4. Sanjay Rajpoot vs Ram Singh on 11 F.txt  (.txt)  size=7429 bytes
 5. SanjayDuttvsTheStateOfHaryana.txt  (.txt)  size=21011 bytes
 6. Vinay_Aggarwal_vs_State_Of_Haryana_on_2_April_2025_1.docx  (.docx)  size=27539 bytes
 7. Vipin_Kumar_vs_Jaydeep_on_21_January_2025_1.docx  (.docx)  size=33532 bytes
 8. Vishal_Shah_vs_Monalisha_Gupta_on_20_February_2025_1.PDF  (.PDF)  size=289474 bytes
 9. Vishnoo_Mittal_vs_M_S_Shakti_Trading_Company_on_17_March_2025_1.PDF  (.PDF)  size=257506 bytes


In [28]:
# 2. Preprocess all files
docs = []
docs_by_id = {}
for i, p in enumerate(files, 1):
    print(f"Preprocessing {i}/{len(files)}: {p.name}")
    doc = preprocess_document(p, doc_id=p.name)
    docs.append(doc)
    docs_by_id[doc['doc_id']] = doc

print('\nPreprocessing finished for all files.')
if docs:
    print_doc_summary(docs[0])

Preprocessing 1/9: BenchOpinion.md
Preprocessing 2/9: indian_supreme_court_judgments.csv
Preprocessing 3/9: Martin_v_Franklin_Capital_Corp.md
Preprocessing 4/9: Sanjay Rajpoot vs Ram Singh on 11 F.txt
Preprocessing 5/9: SanjayDuttvsTheStateOfHaryana.txt
Preprocessing 6/9: Vinay_Aggarwal_vs_State_Of_Haryana_on_2_April_2025_1.docx
Preprocessing 7/9: Vipin_Kumar_vs_Jaydeep_on_21_January_2025_1.docx
Preprocessing 8/9: Vishal_Shah_vs_Monalisha_Gupta_on_20_February_2025_1.PDF
Preprocessing 9/9: Vishnoo_Mittal_vs_M_S_Shakti_Trading_Company_on_17_March_2025_1.PDF

Preprocessing finished for all files.

Document: BenchOpinion.md
----------------------------------------------------------------------------------------------------
Path: data\BenchOpinion.md
Tokens: 817, Sentences: 34, Paragraphs: 4

Tokens preview:
bench opinion october term 2005 syllabus note where it is feasible a syllabus headnote will be released content preserved exactly as provided bench opinion october term 2005 1 syllabus 

In [29]:
print('\nPreprocessing complete. Example document keys and sizes:')
for doc in docs:
    print(f" - {doc['doc_id']}: tokens={len(doc['tokens'])}, sentences={len(doc['sentences'])}, paragraphs={len(doc['paragraphs'])}")

print('\nShow detailed summary for first 2 documents:')
for doc in docs[:2]:
    print_doc_summary(doc, tokens_preview=50)


Preprocessing complete. Example document keys and sizes:
 - BenchOpinion.md: tokens=817, sentences=34, paragraphs=4
 - indian_supreme_court_judgments.csv: tokens=5191404, sentences=164023, paragraphs=1
 - Martin_v_Franklin_Capital_Corp.md: tokens=870, sentences=55, paragraphs=26
 - Sanjay Rajpoot vs Ram Singh on 11 F.txt: tokens=1287, sentences=57, paragraphs=1
 - SanjayDuttvsTheStateOfHaryana.txt: tokens=3525, sentences=129, paragraphs=1
 - Vinay_Aggarwal_vs_State_Of_Haryana_on_2_April_2025_1.docx: tokens=1966, sentences=66, paragraphs=1
 - Vipin_Kumar_vs_Jaydeep_on_21_January_2025_1.docx: tokens=4754, sentences=209, paragraphs=1
 - Vishal_Shah_vs_Monalisha_Gupta_on_20_February_2025_1.PDF: tokens=6866, sentences=424, paragraphs=1
 - Vishnoo_Mittal_vs_M_S_Shakti_Trading_Company_on_17_March_2025_1.PDF: tokens=2505, sentences=77, paragraphs=1

Show detailed summary for first 2 documents:

Document: BenchOpinion.md
-------------------------------------------------------------------------

In [30]:
# 3. Build positional inverted index
index = build_positional_inverted_index(docs)

# 4. Display sorted inverted index (terms in alphabetical order)
sorted_index = sorted_inverted_index_repr(index)
print('\n--- Inverted Index (first 60 terms with document frequency and sample postings) ---')
count = 0
for term, postings in sorted_index.items():
    df = len(postings)
    # show a small sample of postings (up to 3 docs)
    sample_postings = {doc_id: postings[doc_id] for doc_id in list(postings)[:3]}
    print('\n' + '-'*80)
    print(f"Term: '{term}' (df={df})")
    print('Sample postings:')
    pp.pprint(sample_postings)
    count += 1
    if count >= 60:
        break
print('\n' + '-'*80)
print(f"... (vocab size = {len(sorted_index)})\n")

# 5. Proximity queries
print('\n=== Example Proximity Queries ===')


--- Inverted Index (first 60 terms with document frequency and sample postings) ---

--------------------------------------------------------------------------------
Term: '0' (df=1)
Sample postings:
{'indian_supreme_court_judgments.csv': [139316,
                                        148309,
                                        329832,
                                        329951,
                                        331472,
                                        331591,
                                        355617,
                                        355619,
                                        355621,
                                        355622,
                                        355623,
                                        370317,
                                        370319,
                                        370321,
                                        370322,
                                        370323,
                               

In [31]:
from IPython.display import display, HTML
q1 = ("counsel", "/10", "ineffective")
if re.match(r"/\d+", q1[1]):
    n = int(q1[1][1:])
    res = proximity_within_n(index, q1[0].lower(), q1[2].lower(), n)
    print('\n' + '='*100)
    print(f"Query: {q1[0]} {q1[1]} {q1[2]} -> Matches: {len(res)}")
    print('-'*100)
    if not res:
        print('No matches found for this proximity query.')
    for doc_id, pairs in res:
        doc = docs_by_id[doc_id]
        for pair in pairs[:3]:
            highlighted, sidx, pidx, token_pos = build_highlighted_snippet(doc, pair)
            print(f"Doc: {doc_id} | sentence_index={sidx} | paragraph_index={pidx} | token_start_pos={token_pos} | pair={pair}")
            display(HTML(f"<div style='font-family: monospace; background:#1f1f1f; color:#e6e6e6; padding:10px; border-radius:6px; margin:6px 0;'>{highlighted}</div>"))
            print()
    print('='*100)


Query: counsel /10 ineffective -> Matches: 1
----------------------------------------------------------------------------------------------------
Doc: indian_supreme_court_judgments.csv | sentence_index=67482 | paragraph_index=0 | token_start_pos=2161469 | pair=(2161476, 2161469)


In [32]:
res_s = proximity_same_sentence(index, docs_by_id, "due", "process")
print('\n' + '='*100)
print(f"Query: due /s process -> Matches: {len(res_s)}")
print('-'*100)
if not res_s:
    print('No sentence-level matches found.')
for doc_id, sidx, pair in res_s:
    doc = docs_by_id[doc_id]
    if pair is not None:
        highlighted, s2, p2, token_pos = build_highlighted_snippet(doc, pair)
        print(f"Doc: {doc_id} | sentence_index={sidx} | paragraph_index={p2} | token_start_pos={token_pos}")
        display(HTML(f"<div style='font-family: monospace; background:#1f1f1f; color:#e6e6e6; padding:10px; border-radius:6px; margin:6px 0;'>{highlighted}</div>"))
    else:
        sent_text = snippet_from_sent_idx(doc, sidx)
        pidx = doc.get('pos_to_para', {}).get(next(iter(index.get('due',{}).get(doc_id,[])), None))
        print(f"Doc: {doc_id} | sentence_index={sidx} | paragraph_index={pidx}")
        print(f"Sentence: {sent_text}\n")
    print()
print('='*100)


Query: due /s process -> Matches: 59
----------------------------------------------------------------------------------------------------
Doc: indian_supreme_court_judgments.csv | sentence_index=1812 | paragraph_index=0 | token_start_pos=44515



Doc: indian_supreme_court_judgments.csv | sentence_index=2938 | paragraph_index=0 | token_start_pos=81251



Doc: indian_supreme_court_judgments.csv | sentence_index=3164 | paragraph_index=0 | token_start_pos=89897



Doc: indian_supreme_court_judgments.csv | sentence_index=15274 | paragraph_index=0 | token_start_pos=430321



Doc: indian_supreme_court_judgments.csv | sentence_index=15276 | paragraph_index=0 | token_start_pos=430537


...


Doc: indian_supreme_court_judgments.csv | sentence_index=163880 | paragraph_index=0 | token_start_pos=5186982



Doc: indian_supreme_court_judgments.csv | sentence_index=163886 | paragraph_index=0 | token_start_pos=5187206



Doc: indian_supreme_court_judgments.csv | sentence_index=163932 | paragraph_index=0 | token_start_pos=5188387



Doc: indian_supreme_court_judgments.csv | sentence_index=163998 | paragraph_index=0 | token_start_pos=5190767



Doc: indian_supreme_court_judgments.csv | sentence_index=164004 | paragraph_index=0 | token_start_pos=5190991


In [33]:
# Example: child /p visitation (short output)
res_p = proximity_same_paragraph(index, docs_by_id, "child", "visitation")
print('\n=== child /p visitation -> Matches: {} ==='.format(len(res_p)))
if not res_p:
    print('No paragraph-level matches found.')
for doc_id, pidx, pair in res_p:
    doc = docs_by_id[doc_id]
    if pair is not None:
        # get a short plain-text snippet around the matched positions
        snippet = extract_snippet_from_positions(doc, pair, window_tokens=4)
        snippet_brief = (snippet[:120].strip() + '...') if len(snippet) > 120 else snippet.strip()
        print(f"{doc_id} | para={pidx} | snippet: {snippet_brief}")
    else:
        para_text = snippet_from_para_idx(doc, pidx)
        para_brief = (para_text[:120].strip() + '...') if len(para_text) > 120 else para_text.strip()
        print(f"{doc_id} | para={pidx} | paragraph-start: {para_brief}")
print('=== End ===')


=== child /p visitation -> Matches: 1 ===
indian_supreme_court_judgments.csv | para=0 | snippet: of the protection of children from sexual offences act 2012 this provision reads as follows 2 1 d child means any person...
=== End ===


In [34]:
from IPython.display import display, HTML
print('\n=== Example Phrase Queries ===')
phrases = ["right to counsel", "due process"]
for ph in phrases:
    res_ph = phrase_query(index, docs_by_id, ph)
    print('\n' + '='*100)
    print(f"Phrase: \"{ph}\" -> Matches: {len(res_ph)}")
    print('-'*100)
    if not res_ph:
        print('No matches found for this phrase.')
    for doc_id, start_pos, sent_idx, para_idx, highlighted in res_ph:
        print(f"Doc: {doc_id} | sentence_index={sent_idx} | paragraph_index={para_idx} | token_start_pos={start_pos}")
        display(HTML(f"<div style='font-family: monospace; background:#1f1f1f; color:#e6e6e6; padding:10px; border-radius:6px; margin:6px 0;'>{highlighted}</div>"))
        print()
    print('='*100)


=== Example Phrase Queries ===

Phrase: "right to counsel" -> Matches: 0
----------------------------------------------------------------------------------------------------
No matches found for this phrase.

Phrase: "due process" -> Matches: 1
----------------------------------------------------------------------------------------------------
Doc: indian_supreme_court_judgments.csv | sentence_index=23991 | paragraph_index=0 | token_start_pos=776718
